In [ ]:

import numpy as np
import netCDF4
import matplotlib.pyplot as plt
from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from scipy import stats
import time
#import numexpr as ne
import xarray as xr
rEarth = 6371220. #in m ##  get from file variable #f_in.sphere_radius
from netCDF4 import Dataset

In [ ]:
##
#find clusters from initial file
# select density layer global data
path='/global/cscratch1/sd/garanaik/e3sm_scratch/cori-knl/'
den=1028 # input which layer required for filter
path_ini='/global/cscratch1/sd/garanaik/data/'
data_pt_ini = xr.open_dataset(path_ini+'particles_17011_18to6_16000_dt30min_test_41nb_jan_jul_culled.nc')
itemindex = np.where(data_pt_ini.buoyancyParticle[0,:]==den)
n=itemindex[0]
np1=(n[0]);npm1=(n[-1]) # min and max index for particular buoyancy layer
index=np.arange(np1,npm1) #index of all particles belonging to density layer
print(index.shape)  #3281952 84y, 1027d,  #3281952 85y, 1027d,

data_den_ini=data_pt_ini.sel(nParticles=slice((n[0]),(n[-1])))  


llon=data_den_ini.lonParticle[:,:].T
llat=data_den_ini.latParticle[:,:].T


### cluster grid points # either selected domain or global
x,y=np.meshgrid(np.linspace(-90,-20,71),np.linspace(20,50,31)) #cluster center at 1 degree
x=np.ravel(np.deg2rad(x)); y=np.ravel(np.deg2rad(y))


In [ ]:
#cluster, circular
radius=100000# 100km= 100,000 m
import time
t1 = time.time()
radiustree = radius / (rEarth * np.sin(np.maximum(np.abs(np.min(y)),np.abs(np.max(y)))))  #y is lat, x is long
#radiustree = radius / (rEarth * np.sin(np.abs(45)))
print(time.time()-t1)
print(radiustree)


t1 = time.time()
from scipy.spatial import cKDTree as KDTree
allparticles = KDTree(np.vstack((llon[:,0],llat[:,0])).T) #llon[:,0] nparticles,time
#print(time.time()-t1)
search = KDTree(np.vstack((x,y)).T)                       # x,y cluster centers  defined...
#print(time.time()-t1)
clusters = search.query_ball_tree(allparticles, radiustree)
print(time.time()-t1) 


Nclusters = x.ravel().shape[0]
Nclusters

In [ ]:
# convert to lat lon in degree
def latlon_from_xyz(xp,yp,zp,r=rEarth):
    rinv=1/r 
    #plat=np.rad2deg(np.arcsin(zp/ np.sqrt(xp**2 + yp**2 + zp**2)))
    plat=(np.rad2deg(np.arcsin(zp*rinv)))
    plon=(np.rad2deg(np.arctan2(yp, xp)))
    return plat, plon

def normalized_haversine_formula(phi1, phi2, lam1, lam2,r=rEarth):
    
    #phi2=np.deg2rad(phi2)
    #phi1=np.deg2rad(phi1)
    #lam1=np.deg2rad(lam1)
    #lam2=np.deg2rad(lam2)
    
    dphi =( phi2 - phi1)
    dlam = (lam2 - lam1)

    a = np.sin(dphi/2.0)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlam/2.0)**2
    c = r*2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0-a))

    return c

def spherical_bearing(phi1, phi2, lam1, lam2): 
     
    #phi2=np.deg2rad(phi2)
    #phi1=np.deg2rad(phi1)
    #lam1=np.deg2rad(lam1)
    #lam2=np.deg2rad(lam2)
    
    dphi = (phi2 - phi1)
    dlam = (lam2 - lam1)

    return np.arctan2(np.sin(dlam)*np.cos(phi2), np.cos(phi1)*np.sin(phi2) - np.sin(phi1)*np.cos(phi2)*np.cos(dlam)) #}}}

def signed_distances(phi1, phi2, lam1, lam2, r=rEarth):  #{{{
  
    dx = normalized_haversine_formula(phi1, phi1, lam1, lam2,r )
    dy = normalized_haversine_formula(phi1, phi2, lam1, lam1,r )
    # fix orientation of points
    bearing = spherical_bearing(phi1, phi2, lam1, lam2)
    # because arctan2 returns results from -pi to pi for bearing, flip values to get right sign
    dx -= 2*dx*(bearing < 0)
    dy -= 2*dy*(abs(bearing) > np.pi/2.0)
    ux = dx/(24.*60.*60.*1)  #m/s
    uy = dy/(24.*60.*60.*1)  #m/s
   
    return ux, uy #}}}  #velocity with dt 1day


def bootstrap_ci(data,rep):
    n=len(data)
    xb = np.random.choice(data, (n, rep), replace=True)
    yb = 1/np.arange(1, n+1)[:, None] * np.cumsum(xb, axis=0)
    upper, lower = np.percentile(yb, [2.5, 97.5], axis=1)
    
    
    return np.nanmean(upper),np.nanmean(lower),np.nanmean(yb)
 
def cluster_mean_dispersion(plat,plon,r):   #here plat, plon corresponds to that of all 
                                             #particles in one cluster, one realization, one time step, one layer
    clat=np.nanmean(plat)
    clon=np.nanmean(plon)
    
    
    dx = normalized_haversine_formula(clat, clat, clon, plon, r)
    dy = normalized_haversine_formula(clat, plat, clon, clon, r)
    dr = normalized_haversine_formula(clat, plat, clon, plon, r)
    
    bearing = spherical_bearing(clat, plat, clon, plon)
    dx -= 2*dx*(bearing < 0)
    dy -= 2*dy*(np.fabs(bearing) > np.pi/2.0)
    
    dxdx_sum = np.sum(dx*dx)/(len(plat)-1)
    dydy_sum = np.sum(dy*dy)/(len(plat)-1)
    dxdy_sum = np.sum(dx*dy)/(len(plat)-1)
    drdr_sum = np.sum(dr*dr)/(len(plat)-1)
    
    return clon,clat,dxdx_sum,dydy_sum,dxdy_sum,drdr_sum,len(plat)   

In [ ]:
#read all realizations_ all particles
#each file corresponds to each realization of 6 month data
# this part can be combined to one dataset with all realzations, time, particles.
# For now I have kept the realizations fies separate
#

path='/global/cscratch1/sd/garanaik/e3sm_scratch/cori-knl/'
#data_80=xr.open_mfdataset(path+
#    r'E3SM_pio2_one-year_test_oRRS18to6v3_pt_16000_4096_4096_256_Ldt30min_41nb_culled_withrestart_from80yrspunup/'
#    r'run/analysis_members/*.nc',concat_dim="Time")

data=xr.open_mfdataset(path+
   r'E3SM_pio2_one-year_test_oRRS18to6v3_pt_16000_4096_4096_256_Ldt30min_41nb_culled_withrestart_from81yr/'
   r'run/analysis_members/*.nc',concat_dim="Time",combine='nested')

data_den=data.sel(nParticles=slice((n[0]),(n[-1])),Time=slice(0,180)) 



# test only loading one realization all particle from one density layer

import time
t1 = time.time()
plat,plon=latlon_from_xyz(data_den.xParticle.values,data_den.yParticle.values,data_den.zParticle.values)
z=data_den.zLevelParticle.values
time.time() - t1  



plat_r=np.deg2rad(plat)
plon_r=np.deg2rad(plon)

import time
t1 = time.time()
uvel,vvel   = signed_distances(plat_r[:-1,:], plat_r[1:,:],plon_r[:-1,:], plon_r[1:,:])
print(uvel.shape, vvel.shape)
time.time() - t1  

llon_allt=plon_r[:,:].T
llat_allt=plat_r[:,:].T
zt=z[:,:].T
u_allt=uvel[:,:].T
v_allt=vvel[:,:].T

Ntime=data_den.xParticle[:,0].shape[0]
Nparticles=data_den.xParticle[0,:].shape[0]
Nclusters = x.ravel().shape[0]



mux      = np.zeros((Ntime,Nclusters))
muy      = np.zeros((Ntime,Nclusters))
dxdx     = np.zeros((Ntime,Nclusters))
dydy     = np.zeros((Ntime,Nclusters))
dxdy     = np.zeros((Ntime,Nclusters))
drdr     = np.zeros((Ntime,Nclusters))
Npart    = np.zeros((Ntime,Nclusters))
depth    = np.zeros((Ntime,Nclusters))
urms     = np.zeros((Ntime,Nclusters))
vrms     = np.zeros((Ntime,Nclusters))
umean    = np.zeros((Ntime,Nclusters))
vmean    = np.zeros((Ntime,Nclusters))


for t in np.arange(Ntime-1):
    #print("t=",t)
    t1 = time.time()
    for c in np.arange(Nclusters):
        t2 = time.time()
        #print("Nclusters=",c)    
        ind=clusters[c]
        if (len(ind)>0):
            #print(t,c,len(ind))
            p_lat=llat_allt[ind,t]
            p_lon=llon_allt[ind,t]
            p_u=u_allt[ind,t]
            p_v=v_allt[ind,t]
            #uvel,vvel=signed_distances(llat_allt[ind,t], llat_allt[ind,t+1],llon_allt[ind,t], llat_allt[ind,t+1])
            mux[t,c],muy[t,c], dxdx[t,c],dydy[t,c],dxdy[t,c],drdr[t,c], Npart[t,c]=cluster_mean_dispersion(p_lat,p_lon,r=rEarth)
            
            urms[t,c]=np.nanstd(p_u)
            vrms[t,c]=np.nanstd(p_v)
            umean[t,c]=np.nanmean(p_u)
            vmean[t,c]=np.nanmean(p_v)
            depth[t,c]=np.nanmean(zt[ind,t])
       
        #print("time per cluster=",time.time()-t2)
    print(t,"time per time=",time.time()-t1)    #time per time= 0.22664380073547363  for 49 cluster
    
import _pickle as pickle
pickle.dump(mux,open("./gulfstream/mux_81_1028.p","wb"))
pickle.dump(muy,open("./gulfstream/muy_81_1028.p","wb"))
pickle.dump(dxdx,open("./gulfstream/dxdx_81_1028.p","wb"))
pickle.dump(dydy,open("./gulfstream/dydy_81_1028.p","wb"))
pickle.dump(dxdy,open("./gulfstream/dxdy_81_1028.p","wb"))
pickle.dump(drdr,open("./gulfstream/drdr_81_1028.p","wb"))
pickle.dump(Npart,open("./gulfstream/Npart_81_1028.p","wb"))
pickle.dump(depth,open("./gulfstream/depth_81_1028.p","wb"))
pickle.dump(urms,open("./gulfstream/urms_81_1028.p","wb"))
pickle.dump(vrms,open("./gulfstream/vrms_81_1028.p","wb"))
pickle.dump(umean,open("./gulfstream/umean_81_1028.p","wb"))
pickle.dump(vmean,open("./gulfstream/vmean_81_1028.p","wb"))   





In [ ]:
#############################################